## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mount Gambier,AU,-37.8333,140.7667,67.55,44,5,1.81,clear sky
1,1,Kalemie,CD,-5.9475,29.1947,70.02,88,100,6.38,light rain
2,2,Fortuna,US,40.5982,-124.1573,61.36,68,90,1.01,overcast clouds
3,3,Alamos,MX,27.0167,-108.9333,78.71,52,100,3.67,overcast clouds
4,4,Turukhansk,RU,65.8167,87.9833,5.25,97,95,3.40,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Alamos,MX,27.0167,-108.9333,78.71,52,100,3.67,overcast clouds
6,6,Cabo San Lucas,MX,22.8909,-109.9124,81.70,76,75,5.01,broken clouds
13,13,Agua Dulce,MX,18.1333,-94.1333,80.38,75,9,8.05,clear sky
14,14,Harper,LR,4.3750,-7.7169,77.45,87,96,7.90,overcast clouds
18,18,Port Blair,IN,11.6667,92.7500,81.73,78,54,4.29,broken clouds
27,27,Sola,VU,-13.8833,167.5500,80.87,74,16,14.25,few clouds
35,35,Vaini,TO,-21.2000,-175.2000,80.76,83,40,11.50,scattered clouds
38,38,Puerto Penasco,MX,31.3167,-113.5333,75.90,78,4,1.01,clear sky
39,39,Roebourne,AU,-20.7833,117.1333,77.72,52,0,8.32,clear sky
46,46,Muisne,EC,0.6000,-80.0333,76.01,79,68,4.45,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                190
City                   190
Country                190
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Alamos,MX,78.71,overcast clouds,27.0167,-108.9333,
6,Cabo San Lucas,MX,81.70,broken clouds,22.8909,-109.9124,
13,Agua Dulce,MX,80.38,clear sky,18.1333,-94.1333,
14,Harper,LR,77.45,overcast clouds,4.3750,-7.7169,
18,Port Blair,IN,81.73,broken clouds,11.6667,92.7500,
27,Sola,VU,80.87,few clouds,-13.8833,167.5500,
35,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
38,Puerto Penasco,MX,75.90,clear sky,31.3167,-113.5333,
39,Roebourne,AU,77.72,clear sky,-20.7833,117.1333,
46,Muisne,EC,76.01,broken clouds,0.6000,-80.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df =  hotel_df[ hotel_df['Hotel Name'] != '']

In [10]:
clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Alamos,MX,78.71,overcast clouds,27.0167,-108.9333,Luz del Sol
6,Cabo San Lucas,MX,81.70,broken clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
13,Agua Dulce,MX,80.38,clear sky,18.1333,-94.1333,Hotel Victoria
14,Harper,LR,77.45,overcast clouds,4.3750,-7.7169,Screensaver
18,Port Blair,IN,81.73,broken clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
27,Sola,VU,80.87,few clouds,-13.8833,167.5500,Leumerus Bungalows
35,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
38,Puerto Penasco,MX,75.90,clear sky,31.3167,-113.5333,Playa Bonita Resort
39,Roebourne,AU,77.72,clear sky,-20.7833,117.1333,Latitude20 Roebourne Village
46,Muisne,EC,76.01,broken clouds,0.6000,-80.0333,Quinta Samay


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd> 
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))